In [11]:
import pandas as pd
import json
import os

import warnings
warnings.filterwarnings('ignore')

In [12]:
params = [
    "base_solution", "heuristic",
    "search", "iterations", "temp_coef", "population_size",
    "mutation_probability","tabu_queue_coef",
    "swap_any_to_generate", "TotalDistance",
    "TotalTime", "UnusedTrucksCount", "OrphansCount", "execution_time",
    "TotalFuel"
]

def generate_data(folder):
    df_data = {key: [] for key in params}
    df_data["TrucksCount"] = []
    for output in os.listdir(folder):
        with open(f"{folder}/{output}") as f:
            json_data = json.load(f)
        df_data["TrucksCount"].append(len(json_data["Trucks"]))
        for key in params:
            if key not in json_data:
                df_data[key].append(None)
            else:
                df_data[key].append(json_data[key])
    return pd.DataFrame(data=df_data)

df = generate_data("results_lviv")

In [13]:
df.to_csv("lviv_results.csv")

In [14]:
df = pd.read_csv("lviv_results.csv")
df.head()

,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
0,0,SIMPLE_GREEDY,ANY_EXPORTATION_PROB1,TABU,500.0,NaN,NaN,NaN,0.5,NaN,1822.1130,4.910227,0,0,71.572512,455.528250,20
1,1,SIMPLE_GREEDY,MOVE_IN_EXPORTATION,ANNEALING,5000.0,0.00001,NaN,NaN,NaN,NaN,1741.2769,12.861653,0,0,338.529538,435.319225,10
2,2,SIMPLE_GREEDY,REMOVE_LOOPS_SWAP_ANY_EXPORTATION,GENETIC,200.0,NaN,10.0,0.1,NaN,NaN,1805.0859,4.795727,0,0,128.327872,451.271475,20
3,3,SIMPLE_GREEDY,REMOVE_LOOPS_SWAP_ANY_EXPORTATION,GENETIC,400.0,NaN,10.0,0.3,NaN,NaN,1777.5801,6.672147,0,0,219.698072,444.395025,15
4,4,SIMPLE_GREEDY,REMOVE_LOOPS_MOVE_EXPORTATION,ANNEALING,3000.0,0.00001,NaN,NaN,NaN,NaN,1803.9392,4.799982,0,0,265.007310,450.984800,20


In [15]:
df[(df["search"] == "TABU") | (df["search"] == "NO_SEARCH")]

,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
0,0,SIMPLE_GREEDY,ANY_EXPORTATION_PROB1,TABU,500.0,NaN,NaN,NaN,0.5,NaN,1822.1130,4.910227,0,0,71.572512,455.528250,20
6,6,SIMPLE_GREEDY,MOVE_IN_EXPORTATION,TABU,200.0,NaN,NaN,NaN,0.5,NaN,1743.8116,12.861793,0,0,68.523277,435.952900,10
7,7,SIMPLE_GREEDY,REMOVE_LOOPS_SWAP_ANY_EXPORTATION,TABU,500.0,NaN,NaN,NaN,0.3,500.0,1744.0947,12.837842,0,0,210.068717,436.023675,10
10,10,SIMPLE_GREEDY,ANY_EXPORTATION_PROB1,TABU,500.0,NaN,NaN,NaN,0.3,NaN,1753.9201,12.956816,0,0,78.126473,438.480025,10
11,11,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,TABU,500.0,NaN,NaN,NaN,0.5,NaN,1798.4006,4.738413,0,0,107.704016,449.600150,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,SIMPLE_GREEDY,MOVE_IN_EXPORTATION,TABU,500.0,NaN,NaN,NaN,0.3,NaN,1805.1405,4.799982,0,0,86.019331,451.285125,20
362,362,SIMPLE_GREEDY,ANY_MOVE_SWAP,TABU,200.0,NaN,NaN,NaN,0.3,NaN,1744.8424,12.915273,0,0,70.034316,436.210600,10
363,363,SIMPLE_GREEDY,ANY_EXPORTATION_PROB1,TABU,500.0,NaN,NaN,NaN,0.3,NaN,1796.0313,6.905716,0,0,78.640221,449.007825,15
370,370,SIMPLE_GREEDY,ANY_EXPORTATION,TABU,200.0,NaN,NaN,NaN,0.5,NaN,1792.0721,6.891307,0,0,65.885711,448.018025,15


In [16]:
def get_plot_data(df, search, trucks_count, base_solution="SIMPLE_GREEDY"):
    return df[((df["search"] == search) | (df["search"] == "NO_SEARCH")) &
              (df["TrucksCount"] == trucks_count) &
              (df["base_solution"] == base_solution)
             ]

tabu_greedy_10 = get_plot_data(df, "TABU", 10)
tabu_greedy_15 = get_plot_data(df, "TABU", 15)
tabu_greedy_20 = get_plot_data(df, "TABU", 20)
annealing_greedy_10 = get_plot_data(df, "ANNEALING", 10)
annealing_greedy_15 = get_plot_data(df, "ANNEALING", 15)
annealing_greedy_20 = get_plot_data(df, "ANNEALING", 20)
genetic_greedy_10 = get_plot_data(df, "GENETIC", 10)
genetic_greedy_15 = get_plot_data(df, "GENETIC", 15)
genetic_greedy_20 = get_plot_data(df, "GENETIC", 20)

In [17]:
import altair as alt
import numpy as np

def plot_results(data, tooltips, title=""):
    title = alt.TitleParams(
        title,
        fontSize=20,
        anchor="middle",
    )

    minDistance = data["TotalDistance"].min()
    maxDistance = data[data["search"] == "NO_SEARCH"]["TotalDistance"].max()
    data = data[data["TotalDistance"] <= maxDistance]
    minTime = data[data["search"] == "NO_SEARCH"]["execution_time"].min()
    maxTime = data["execution_time"].max()

    data["Relation"] = (maxDistance - data["TotalDistance"]) / (data["execution_time"] - minTime + 0.00001)

    y = np.arange(minTime - 10, maxTime + 10, 0.1)
    best_relation = data["Relation"].max()
    y = y[- best_relation * (y - minTime) + maxDistance < maxDistance + 10]
    y = y[- best_relation * (y - minTime) + maxDistance > minDistance - 10]
    source = pd.DataFrame({
      'x': - best_relation * (y - minTime) + maxDistance,
      'y': y
    })

    best_relation_line = alt.Chart(source).mark_line().encode(
        x=alt.X('x', scale=alt.Scale(domain=[maxDistance + 10, minDistance - 10])),
        y=alt.Y("y")
    )

    points = alt.Chart(data, title=title).mark_circle(
        size=60
    ).encode(
        x=alt.X(
            "TotalDistance",
            scale=alt.Scale(domain=[maxDistance + 10, minDistance - 10]),
            title="Total distance"
        ),
        y=alt.Y(
            "execution_time",
            scale=alt.Scale(domain=[minTime - 10, maxTime + 10]),
            title="Execution time"
        ),
        color=alt.Color("search", title="Search"),
        tooltip=tooltips
    )

    return points + best_relation_line


In [18]:
tabu_tooltips = [
    alt.Tooltip("heuristic", title="Heuristic"),
    alt.Tooltip("search", title="Search"),
    alt.Tooltip("base_solution", title="Base solution"),
    alt.Tooltip("iterations", title="Iterations"),
    alt.Tooltip("TotalDistance", title="Total distance"),
    alt.Tooltip("execution_time", title="Execution time"),
    alt.Tooltip("tabu_queue_coef", title="Queue coef"),
]

annealing_tooltips = [
    alt.Tooltip("heuristic", title="Heuristic"),
    alt.Tooltip("search", title="Search"),
    alt.Tooltip("base_solution", title="Base solution"),
    alt.Tooltip("iterations", title="Iterations"),
    alt.Tooltip("TotalDistance", title="Total distance"),
    alt.Tooltip("execution_time", title="Execution time"),
    alt.Tooltip("temp_coef", title="Temp coef"),
]

genetic_tooltips = [
    alt.Tooltip("heuristic", title="Heuristic"),
    alt.Tooltip("search", title="Search"),
    alt.Tooltip("base_solution", title="Base solution"),
    alt.Tooltip("iterations", title="Iterations"),
    alt.Tooltip("TotalDistance", title="Total distance"),
    alt.Tooltip("execution_time", title="Execution time"),
    alt.Tooltip("population_size", title="Population size"),
    alt.Tooltip("mutation_probability", title="Mutation probability"),
]


In [19]:
plot_results(tabu_greedy_10, tabu_tooltips, "Tabu results for 10 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/tabu_10.html")

In [20]:
plot_results(annealing_greedy_10, annealing_tooltips, "Simulated annealing results for 10 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/sa_10.html")

In [21]:
plot_results(genetic_greedy_10, genetic_tooltips, "Genetic algorithm results for 10 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/ga_10.html")

In [22]:
plot_results(tabu_greedy_15, tabu_tooltips, "Tabu results for 15 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/tabu_15.html")

In [23]:
plot_results(annealing_greedy_15, annealing_tooltips, "Simulated annealing results for 15 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/sa_15.html")

In [24]:
plot_results(genetic_greedy_15, genetic_tooltips, "Genetic algorithm results for 15 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/ga_15.html")

In [25]:
plot_results(tabu_greedy_20, tabu_tooltips, "Tabu results for 20 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/tabu_20.html")

In [26]:
plot_results(annealing_greedy_20, annealing_tooltips, "Simulated annealing results for 20 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/sa_20.html")

In [27]:
plot_results(genetic_greedy_20, genetic_tooltips, "Genetic algorithm results for 20 trucks").properties(
    width=600, height=400, background="#F9F9F9"
).save("img/ga_20.html")

In [28]:
df[df["TrucksCount"] == 10].nsmallest(15, columns=["TotalDistance"])

,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
219,219,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1720.6410,12.562769,0,0,399.258123,430.160250,10
122,122,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.3,NaN,NaN,1721.2374,12.616251,0,0,708.648684,430.309350,10
261,261,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1721.3918,12.596487,0,0,904.773605,430.347950,10
225,225,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1721.7172,12.547920,0,0,539.926068,430.429300,10
276,276,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1722.1398,12.480627,0,0,651.631356,430.534950,10
361,361,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.3,NaN,NaN,1722.2792,12.578844,0,0,386.875207,430.569800,10
99,99,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,ANNEALING,5000.0,0.00001,NaN,NaN,NaN,NaN,1723.3443,12.721680,0,0,489.911571,430.836075,10
177,177,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,200.0,NaN,20.0,0.1,NaN,NaN,1728.1472,12.618764,0,0,362.907387,432.036800,10
167,167,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,ANNEALING,3000.0,0.00001,NaN,NaN,NaN,NaN,1728.7224,12.749340,0,0,317.046356,432.180600,10
320,320,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,200.0,NaN,10.0,0.1,NaN,NaN,1730.2710,12.635393,0,0,223.284241,432.567750,10


In [29]:
df[df["TrucksCount"] == 15].nsmallest(15, columns=["TotalDistance"])

,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
184,184,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1762.3934,6.585204,0,0,588.605112,440.598350,15
207,207,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1763.0398,6.564831,0,0,1258.932826,440.759950,15
153,153,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.3,NaN,NaN,1764.0461,6.565382,0,0,802.709540,441.011525,15
145,145,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1765.5348,6.610611,0,0,395.736161,441.383700,15
176,176,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,ANNEALING,5000.0,0.00001,NaN,NaN,NaN,NaN,1766.2468,6.542956,0,0,480.885612,441.561700,15
9,9,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.3,NaN,NaN,1767.2926,6.592040,0,0,400.347700,441.823150,15
193,193,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1768.0011,6.693460,0,0,701.526734,442.000275,15
203,203,SIMPLE_GREEDY,REMOVE_LOOPS_SWAP_ANY_EXPORTATION,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1770.9532,6.581324,0,0,438.421854,442.738300,15
41,41,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,TABU,500.0,NaN,NaN,NaN,0.3,500.0,1770.9661,6.668724,0,0,331.923559,442.741525,15
270,270,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,200.0,NaN,10.0,0.1,NaN,NaN,1770.9704,6.620247,0,0,216.716352,442.742600,15


In [30]:
df[df["TrucksCount"] == 20].nsmallest(15, columns=["TotalDistance"])

,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
129,129,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1788.5737,4.761813,0,0,395.348654,447.143425,20
32,32,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,ANNEALING,5000.0,0.00001,NaN,NaN,NaN,NaN,1791.5387,4.750596,0,0,453.009711,447.884675,20
332,332,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1791.5440,4.724753,0,0,1092.415264,447.886000,20
226,226,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.3,NaN,NaN,1791.7197,4.724753,0,0,707.092381,447.929925,20
271,271,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,ANNEALING,3000.0,0.00001,NaN,NaN,NaN,NaN,1792.1471,4.742800,0,0,324.871405,448.036775,20
92,92,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1793.4086,4.724802,0,0,539.605015,448.352150,20
81,81,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.3,NaN,NaN,1794.1307,4.723376,0,0,396.511053,448.532675,20
101,101,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,TABU,500.0,NaN,NaN,NaN,0.3,500.0,1795.2015,4.667231,0,0,569.968375,448.800375,20
243,243,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,200.0,NaN,20.0,0.1,NaN,NaN,1796.2827,4.742382,0,0,352.215985,449.070675,20
198,198,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,200.0,NaN,10.0,0.1,NaN,NaN,1797.0729,4.741169,0,0,229.174504,449.268225,20


In [31]:
def create_top_data(df, trucks_count, amount):
    top_data = df[df["TrucksCount"] == trucks_count].nsmallest(amount, columns=["TotalDistance"]).reset_index()
    no_search = df[(df["search"] == "NO_SEARCH") &
                   (df["TrucksCount"] == trucks_count) & 
                   (df["base_solution"] == "SIMPLE_GREEDY")]
    return top_data.append(no_search)


top_greedy_10 = create_top_data(df, 10, 15)
top_greedy_15 = create_top_data(df, 15, 15)
top_greedy_20 = create_top_data(df, 20, 15)

In [32]:
top_tooltips = [
    alt.Tooltip("heuristic", title="Heuristic"),
    alt.Tooltip("search", title="Search"),
    alt.Tooltip("base_solution", title="Base solution"),
    alt.Tooltip("iterations", title="Iterations"),
    alt.Tooltip("TotalDistance", title="Total distance"),
    alt.Tooltip("execution_time", title="Execution time"),
    alt.Tooltip("population_size", title="Population size"),
    alt.Tooltip("mutation_probability", title="Mutation probability"),
    alt.Tooltip("temp_coef", title="Temp coef"),
    alt.Tooltip("tabu_queue_coef", title="Queue coef"),
]

In [53]:
plot_results(top_greedy_10, top_tooltips, title="Top results for 10 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [54]:
plot_results(top_greedy_15, top_tooltips, title="Top results for 15 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [55]:
plot_results(top_greedy_20, top_tooltips, title="Top results for 20 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [36]:
def top_by_search(df, trucks_count, amount, search):
    return df[(df["TrucksCount"] == trucks_count) & (df["search"] == search)].nsmallest(
        amount, 
        columns=["TotalDistance"]
    ).reset_index()

top_by_search(df, 10, 5, "GENETIC")

,index,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
0,219,219,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1720.6410,12.562769,0,0,399.258123,430.16025,10
1,122,122,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.3,NaN,NaN,1721.2374,12.616251,0,0,708.648684,430.30935,10
2,261,261,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1721.3918,12.596487,0,0,904.773605,430.34795,10
3,225,225,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1721.7172,12.547920,0,0,539.926068,430.42930,10
4,276,276,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1722.1398,12.480627,0,0,651.631356,430.53495,10


In [180]:
top_by_search(df, 15, 5, "GENETIC")

,index,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
0,184,184,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1762.3934,6.585204,0,0,588.605112,440.598350,15
1,207,207,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1763.0398,6.564831,0,0,1258.932826,440.759950,15
2,153,153,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.3,NaN,NaN,1764.0461,6.565382,0,0,802.709540,441.011525,15
3,145,145,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1765.5348,6.610611,0,0,395.736161,441.383700,15
4,9,9,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.3,NaN,NaN,1767.2926,6.592040,0,0,400.347700,441.823150,15


In [179]:
top_by_search(df, 20, 5, "GENETIC")

,index,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
0,129,129,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION_FIXED2,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1788.5737,4.761813,0,0,395.348654,447.143425,20
1,332,332,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.1,NaN,NaN,1791.5440,4.724753,0,0,1092.415264,447.886000,20
2,226,226,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,20.0,0.3,NaN,NaN,1791.7197,4.724753,0,0,707.092381,447.929925,20
3,92,92,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.1,NaN,NaN,1793.4086,4.724802,0,0,539.605015,448.352150,20
4,81,81,SIMPLE_GREEDY,ANY_MOVE_EXPORTATION,GENETIC,400.0,NaN,10.0,0.3,NaN,NaN,1794.1307,4.723376,0,0,396.511053,448.532675,20


In [57]:
df_groningen = generate_data("data/Groningen/best_results")
df_new_york = generate_data("data/NewYork/best_results")
df_spain = generate_data("data/SantBoiDeLlobregat/best_results")
df_pittsburg = generate_data("data/Pittsburgh/best_results")

In [58]:
def generate_names(df):
    df["Name"] = df["search"].apply(str) + " " + df["swap_any_to_generate"].apply(str).replace("nan", "")
    return df

df_groningen = generate_names(df_groningen)
df_new_york = generate_names(df_new_york)
df_spain = generate_names(df_spain)
df_pittsburg = generate_names(df_pittsburg)

In [84]:
def create_diff_plot(df, title="", width=600, height=400):

    header = alt.TitleParams(
        text=title,
        fontSize=16,
        subtitleFontSize=12,
        anchor="middle",
        subtitlePadding=5
    )

    header_plot = alt.Chart(title=header).mark_text().properties(width=width, height=0)


    tooltip = [
        alt.Tooltip("heuristic", title="Heuristic"),
        alt.Tooltip("search", title="Search"),
        alt.Tooltip("base_solution", title="Base solution"),
        alt.Tooltip("iterations", title="Iterations"),
        alt.Tooltip("TotalDistance", title="Total distance"),
        alt.Tooltip("execution_time", title="Execution time"),
        alt.Tooltip("population_size", title="Population size"),
        alt.Tooltip("mutation_probability", title="Mutation probability"),
        alt.Tooltip("temp_coef", title="Temp coef"),
        alt.Tooltip("tabu_queue_coef", title="Queue coef")
    ]

    top_title = alt.TitleParams(
        text="Total distance difference (in percents)",
        fontSize=12,
    )
    
    new_data = df
    new_data["diff"] = (new_data["TotalDistance"].max() - new_data["TotalDistance"]) / new_data["TotalDistance"].max()
    new_data["diff_time"] = new_data["execution_time"] - new_data["execution_time"].min()
    new_data = new_data[new_data["TotalDistance"] != new_data["TotalDistance"].max()]

    top = alt.Chart(new_data, title=top_title).mark_bar(size=40).encode(
        x=alt.X("Name", axis=alt.Axis(title="", labelFontSize=0)),
        y=alt.Y("diff", title="", axis=alt.Axis(format='%')),
        color=alt.value("lightblue"),
        tooltip=tooltip,
    ).properties(width=width, height=height//2)


    middle =  alt.Chart(new_data).encode(
        x=alt.X("Name", axis=None),
        text=alt.Text("Name"),
    ).mark_text().properties(width=width, height=10)


    bottom_title = alt.TitleParams(
        text="Execution time difference",
        fontSize=12,
        orient="bottom"
    )

    bottom = alt.Chart(new_data, title=bottom_title).mark_bar(size=40).encode(
        x=alt.X("Name", axis=alt.Axis(orient="top", title="", labelFontSize=0)),
        y=alt.Y("execution_time", title="s", scale=alt.Scale(reverse=True)),
        color=alt.value("lightgray"),
        tooltip=tooltip,
    ).properties(width=width, height=height//2)

    return alt.vconcat(header_plot, top, middle, bottom, spacing=0).configure_view(strokeWidth=0)

create_diff_plot(
    df_groningen,
    title="Best algorithms results for Groningen",
    width=600,
    height=400
)

alt.VConcatChart(...)

In [85]:
create_diff_plot(
    df_new_york,
    title="Best algorithms results for New York",
    width=600,
    height=400
)

alt.VConcatChart(...)

In [86]:
create_diff_plot(
    df_pittsburg,
    title="Best algorithms results for Pittsburg",
    width=600, height=400
)

alt.VConcatChart(...)

In [87]:
create_diff_plot(
    df_spain, 
    title="Best algorithms results for Sant Boi de Llobregat",
    width=600,
    height=400
)

alt.VConcatChart(...)